### IPO scraper

In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

from time import sleep

import dill

ModuleNotFoundError: No module named 'dill'

In [ ]:
dill.load_session('ipo_scraper.db')

## 1. Scraping

In [ ]:
dates = pd.period_range('2010-01-01', '2018-12-31', freq='M')
dates

In [ ]:
df = pd.DataFrame()
links = []
url = 'https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=%s'
headers = {'User-Agent' : "non-profit learning project"}

for idx in dates:
    print(f'fill scrap - {url % idx}')
    result = requests.get(url % idx, headers=headers)
    sleep(30)
    content = result.content
    
    if not "There is no data for this month" in str(content):
        table = pd.read_html(content)[0]
        df = df.append(table, ignore_index=True)
    
        soup = BeautifulSoup(content)
    
        m = soup.find_all('a', id=re.compile('two_column_main_content_rptPricing_company_\d'))
        print(f"length of table vs length of links - {table.shape[0]-len(m)}")
        
        for link in m:
            links.append(link['href'])

    elif "There is no data for this month" in str(content):
        pass

In [ ]:
descriptions = []
address = []
employees = []

for lnk in links:
    print("scraping -", lnk)
    res = requests.get(lnk).content
    soup = BeautifulSoup(res)
    desc = [x.get_text() for x in soup.findAll("div", {"class": "ipo-comp-description"})]
    desc2 = [x.get_text() for x in soup.findAll("div", {"id": "read_more_div_toggle1"})]
    if (not desc)|(not desc2):
        descriptions.append('')
    else:
        desc.extend(desc2)
        descriptions.append(desc)
    
    table = pd.read_html(res)
    address.append(table[0][1][1])
    employees.append(table[0][1][5])

In [2]:
print(f"checking length of rows in dataframe {df.shape[0]}, descriptions {len(descriptions)}, employees {len(employees)}, address {len(address)}")

NameError: name 'df' is not defined

In [3]:
descr_pretty = [str(x).replace("\\n', '\\n", "").replace("['\\nCompany Description\\n", "").replace("\\n", " ").replace("']","")
                for x in descriptions]

NameError: name 'descriptions' is not defined

In [4]:
#adding state column
states = []
for addr in address:
    pattern = re.compile(", [A-Z]{2} \d{5}")
    state = pattern.findall(str(addr))
    if state:
        state = str(state).split(" ")[1]
    else:
        state = ''
    states.append(state)

NameError: name 'address' is not defined

In [5]:
employees_clean = []
for emp in employees:
    try:
        employees_clean.append(int(emp))
    except ValueError: 
        employees_clean.append(np.nan)

NameError: name 'employees' is not defined

### Adding new columns:

In [6]:
df['employees'] = employees_clean

NameError: name 'df' is not defined

In [385]:
df['address'] = address
df['address'] = df['address'].fillna('')

In [386]:
df['US_state'] = states

In [387]:
df['descriptions'] = descr_pretty

In [388]:
df['link_nasdaq'] = links

In [389]:
df.head()

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,descriptions,link_nasdaq
0,"CHINA ELECTRIC MOTOR, INC.",CELM,NASDAQ,$4.50,5000000,"$22,500,000",1/29/2010,920.0,"SUNNA MOTOR INDUSTRY PARK, JIAN'ANFUYONG HI-TE...",,"Through Shenzhen YPC, we engage in the design,...",https://www.nasdaq.com/markets/ipos/company/ch...
1,IFM INVESTMENTS LTD,CTC,New York Stock Exchange,$7,12487500,"$87,412,500",1/28/2010,4654.0,"9/A5, EAST WING, HANWEI PLAZANO.7 GUANGHUA ROA...",,We are a leading comprehensive real estate ser...,https://www.nasdaq.com/markets/ipos/company/if...
2,ANDATEE CHINA MARINE FUEL SERVICES CORP,AMCF,NASDAQ,$6.30,3134921,"$19,750,002",1/26/2010,128.0,NO. 68 BINHAI RD DALIAN XIGANG DISTRICTDALIAN ...,,"We, through our VIE entities are engaged in th...",https://www.nasdaq.com/markets/ipos/company/an...
3,CHINA HYDROELECTRIC CORP,CHC,New York Stock Exchange,$16,6000000,"$96,000,000",1/25/2010,336.0,"420 LEXINGTON AVENUESUITE 860NEW YORK, NY 10170",NY,"We are a fast-growing consolidator, operator a...",https://www.nasdaq.com/markets/ipos/company/ch...
4,CHESAPEAKE LODGING TRUST,CHSP,New York Stock Exchange,$20,7500000,"$150,000,000",1/22/2010,3.0,"4300 WILSON BOULEVARDSUITE 625ARLINGTON, VA 22203",VA,We are a self-advised hotel investment company...,https://www.nasdaq.com/markets/ipos/company/ch...


In [588]:
df.shape

(1866, 12)

## 2. Converting Column Types

In [11]:
#transforming column from str type to datetime type
df['Date Priced'] = pd.to_datetime(df['Date Priced'])

NameError: name 'df' is not defined

In [590]:
df['year'] = df['Date Priced'].map(lambda x: x.year)

In [591]:
#converting column from str type to float
df['Price'] = df['Price'].map(lambda x: x.replace("$", ""))
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

In [592]:
df['Offer Amount'] = df['Offer Amount'].map(lambda x: x.replace("$", "").replace(",", ""))
df['Offer Amount'] = pd.to_numeric(df['Offer Amount'], errors='coerce')

In [602]:
df['Shares'] = pd.to_numeric(df['Shares'], errors='coerce')

In [610]:
df.dtypes

Company Name            object
Symbol                  object
Market                  object
Price                  float64
Shares                 float64
Offer Amount           float64
Date Priced     datetime64[ns]
employees              float64
address                 object
US_state                object
descriptions            object
link_nasdaq             object
year                     int64
dtype: object

## 3. Cleaning:

In [595]:
def clean_from_mergers(dataframe):
    acquisition_cond = dataframe['Company Name'].str.contains('ACQUISITION|MERGER', case=False, regex=True)
    blankcheck_cond = dataframe['descriptions'].str.contains('blank check', case=False)
    mergerdesc_cond = dataframe['descriptions'].str.contains('entering into a merger, share exchange, asset acquisition', case=False)
    return dataframe[~acquisition_cond & ~blankcheck_cond & ~mergerdesc_cond].copy()

In [596]:
def clean_from_etfs(dataframe):
    etf_cond1 = dataframe['descriptions'].str.contains('exchange-traded|Fund Equity|investment strategy', case=False, regex=True)
    etf_cond2 = dataframe['Company Name'].str.contains(' ETF')
    trust = dataframe['Company Name'].str.contains(' TRUST')
    realestate = dataframe['descriptions'].str.contains('properties|leasing|real estate|leased|REIT', case=False, regex=True)
    return dataframe[~etf_cond1 & ~etf_cond2 & ~(trust & ~realestate)].copy()              

In [605]:
df_nomergers = clean_from_mergers(df)
print(f"{df.shape[0]-df_nomergers.shape[0]} companies went to IPO with the purpose of merger/acquisition. These companies will be excluded")

166 companies went to IPO with the purpose of merger/acquisition. These companies will be excluded


In [606]:
df_clean = clean_from_etfs(df_nomergers)
print(f"{df_nomergers.shape[0]-df_clean.shape[0]} companies on IPO were ETFs or trusts (but not REITS). These companies will be excluded")

53 companies on IPO were ETFs or trusts (but not REITS). These companies will be excluded


I will also exclude those companies that were not listed on a formal exchange but traded over the counter.

In [607]:
otc = df_clean[df_clean['Market'] == 'OTCBB'].shape[0]
print(f'there are {otc} companies that were traded over the counter. Will be removed')
df_clean = df_clean[~(df_clean['Market'] == 'OTCBB')]

there are 47 companies that were traded over the counter. Will be removed


In [615]:
print(f"the final IPO list includes - {df_clean.shape[0]} companies")

the final IPO list includes - 1600 companies


In [616]:
df_clean.to_csv('ipo_clean_2010_2018.csv', index=False)

In [8]:
dill.dump_session('ipo_scraper.db')